In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/content/fake_or_real_news.csv')

In [3]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [46]:
df['label'].value_counts()

,count
label,
1,3171
0,3164


In [87]:
# prompt: Using dataframe df: i want lebel encoding on df.label

from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Fit and transform the 'label' column
df['label'] = le.fit_transform(df['label'])

# Display the first few rows to show the changes
# print(df.head())
df.head()
# df.drop('Unnamed: 0',axis=1,inplace=True)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1


In [48]:
df.isnull().sum()

,0
title,0
text,0
label,0


In [49]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [50]:
## Get the Dependent features
y=df['label']

In [51]:
X.shape

(6335, 2)

In [52]:
y.shape


(6335,)

In [53]:
import tensorflow as tf

In [88]:
tf.__version__

'2.18.0'

In [54]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [55]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [56]:
messages=X.copy()

In [89]:
### Vocabulary size
voc_size=10000

In [90]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [91]:
corpus

['smell hillari fear',
 'watch exact moment paul ryan commit polit suicid trump ralli video',
 'kerri go pari gestur sympathi',
 'berni support twitter erupt anger dnc tri warn',
 'battl new york primari matter',
 'tehran usa',
 'girl horrifi watch boyfriend left facetim',
 'britain schindler die',
 'fact check trump clinton command chief forum',
 'iran reportedli make new push uranium concess nuclear talk',
 'three clinton iowa glimps fire elud hillari clinton campaign',
 'donald trump shockingli weak deleg game somehow got even wors',
 'strong solar storm tech risk today news oct video',
 'way america prepar world war',
 'trump take cruz lightli',
 'women lead differ',
 'shock michel obama hillari caught glamor date rape promot',
 'hillari clinton huge troubl america notic sick thing hidden pictur liberti writer news',
 'iran bill obama like',
 'chart explain everyth need know partisanship america',
 'slipperi slope trump propos ban muslim',
 'episod sunday wire hail deplor special g

In [92]:
messages

,title,text
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello..."
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T..."
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...
...,...,...
6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...
6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene..."


In [93]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4369, 3801, 490],
 [170, 5409, 4926, 5182, 7429, 6364, 7984, 2061, 5294, 2158, 5198],
 [9870, 9908, 6136, 9186, 8582],
 [3528, 1761, 921, 7403, 4975, 3621, 3673, 8657],
 [8531, 459, 333, 7240, 4779],
 [1111, 8037],
 [9483, 3520, 170, 7913, 8318, 6730],
 [1999, 1852, 738],
 [4899, 660, 5294, 9224, 197, 4056, 1353],
 [3623, 6286, 8722, 459, 3130, 2752, 7346, 5102, 8384],
 [5579, 9224, 2849, 3885, 3780, 6270, 3801, 9224, 5912],
 [9450, 5294, 7392, 7134, 261, 4800, 7521, 6499, 6179, 7097],
 [3988, 4683, 5799, 2252, 2106, 1174, 3147, 6342, 5198],
 [1241, 2740, 9772, 8150, 546],
 [5294, 5562, 4834, 1205],
 [4845, 7627, 9344],
 [6093, 8461, 2210, 3801, 1801, 2482, 8692, 8577, 7150],
 [3801,
  9224,
  1679,
  9567,
  2740,
  3065,
  3173,
  5913,
  6512,
  5786,
  9261,
  5760,
  3147],
 [3623, 9758, 2210, 5294],
 [8084, 788, 9804, 1164, 321, 2734, 2740],
 [9744, 8897, 5294, 5979, 6801, 8031],
 [9904, 3625, 2830, 9726, 6910, 1554, 5670, 5018, 9009],
 [3801, 9224, 8722, 8888, 4035, 1877, 5932

In [94]:
onehot_repr[1]

[170, 5409, 4926, 5182, 7429, 6364, 7984, 2061, 5294, 2158, 5198]

In [95]:
messages.title[1]

'Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)'

In [97]:
sent_length=25
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4369 3801  490]
 [   0    0    0 ... 5294 2158 5198]
 [   0    0    0 ... 6136 9186 8582]
 ...
 [   0    0    0 ... 1965 4136 3930]
 [   0    0    0 ... 1769 3812 2830]
 [   0    0    0 ... 4875 5294 4779]]


In [98]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,  170, 5409, 4926, 5182, 7429, 6364, 7984, 2061,
       5294, 2158, 5198], dtype=int32)

In [99]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       4369, 3801,  490], dtype=int32)

In [101]:
## Creating model
embedding_vector_features=50 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [102]:
len(embedded_docs),y.shape

(6335, (6335,))

In [103]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [105]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 9s 90ms/step - accuracy: 0.5600 - loss: 0.6765 - val_accuracy: 0.7575 - val_loss: 0.5177
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.8408 - loss: 0.3903 - val_accuracy: 0.7881 - val_loss: 0.4755
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 92ms/step - accuracy: 0.9093 - loss: 0.2388 - val_accuracy: 0.7967 - val_loss: 0.5379
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.9374 - loss: 0.1578 - val_accuracy: 0.7905 - val_loss: 0.5971
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 56ms/step - accuracy: 0.9673 - loss: 0.1008 - val_accuracy: 0.7862 - val_loss: 0.7228
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.9808 - loss: 0.0636 - val_accuracy: 0.7791 - val_loss: 0.8865
Epoch 7/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.9862 - loss: 0.0454 - val_accuracy: 0.7805 - val_loss: 0.9978
Epoch 8/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - accuracy: 0.9910 - loss: 0.0301 - val_accuracy: 0.7580 - v

In [106]:
y_final

array([0, 0, 1, ..., 0, 1, 1])

In [124]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [125]:
y_pred=model.predict(X_test)

66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step


In [126]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [127]:
from sklearn.metrics import confusion_matrix

In [128]:
confusion_matrix(y_test,y_pred)

array([[1071,    0],
       [1020,    0]])

In [130]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)